In [1]:
import os, shutil
import pandas as pd
import os
import re
import string
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import statistics
from scipy.stats.mstats import gmean
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
import os
import random
cwd = os.getcwd()

# Importing materials data

In [2]:
materials = pd.read_csv(cwd+"//data//new_data_2.csv")
# Pre-processing materials dataframe
# Namely, dropping Unnamed: 12 and other irrelevant columns
materials = materials.drop(
        columns=[
            "Unnamed: 12", 
            "location mining", 
            "reference mining", 
            "mining assumptions", 
            "location processing", 
            "reference processing",
            "amount (kg/MWh)"]
    )
# Rename columns for convenience
materials = materials.rename(
        columns={"coordinate mining site":"mining",
                 "coordinate manufacturing":"manufacturing",
                 "amount (kg/MW)" : "amount"}
    )
# Dummy data for coordinate manufacturing is going to be China for now or 
materials["manufacturing"] = np.random.choice(["(39.915376, 115.464970)", "(38.395595, 140.401897)"], size=len(materials))
# Drop NaN values, but in end analysis, there shall be none such
materials = materials.dropna()
# Convert mining and manufacturing coordinates into float tuples
materials["manufacturing"] = materials["manufacturing"].apply(lambda tup_s: tuple(float(x) for x in tup_s[1:-1].split(',')))
materials["mining"] = materials["mining"].apply(lambda tup_s: tuple(float(x) for x in tup_s[1:-1].split(',')))

In [3]:
materials

,source,stream,material,amount,mining,manufacturing
0,gas,back_waste,aluminum,48.960000,"(55.339722, -160.497222)","(38.395595, 140.401897)"
1,hydro,back_waste,aluminum,380.450304,"(37.8833, -84.1017)","(38.395595, 140.401897)"
2,coal,back_waste,aluminum,100.560000,"(32.16, -110.9047)","(38.395595, 140.401897)"
3,wind,back_waste,aluminum,2006.700000,"(32.673097, -115.392026)","(38.395595, 140.401897)"
4,solar,back_waste,aluminum,4560.000000,"(33.764741, -118.09913)","(39.915376, 115.46497)"
...,...,...,...,...,...,...
240,wind,front_waste,zinc_mining_w,872.430933,"(68.071989, -162.876044)","(38.395595, 140.401897)"
241,nuclear,front_waste,zinc_mining_w,478.932381,"(68.071989, -162.876044)","(39.915376, 115.46497)"
242,nuclear,oper_need,zirconium,804.168000,"(-30.90926391474549, 132.22041172635542)","(38.395595, 140.401897)"
243,nuclear,oper_waste,zirconium,804.168000,"(38.850283, -120.381883)","(39.915376, 115.46497)"


In [4]:
# Filter for oper_need and front_need, since only these streams are necessary for building/operating a plant
need_materials = materials.loc[materials['stream'].isin(['oper_need', 'front_need'])]
# Calculate tonmile/MW = amount (kg) / 1000 * the distance to transport materials from mining site to the manufacturing
need_materials["mining tonmile/MW"] = (need_materials["amount"] / 1000) * need_materials.apply(lambda row: geodesic(row["mining"], row["manufacturing"]).miles, axis=1)
need_materials = need_materials.drop(columns=["mining"])
# need_materials = need_materials.groupby(["source", "stream", "material"]).sum(["mining_dist"])
need_materials

/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_61461/2942254522.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_materials["mining tonmile/MW"] = (need_materials["amount"] / 1000) * need_materials.apply(lambda row: geodesic(row["mining"], row["manufacturing"]).miles, axis=1)


,source,stream,material,amount,manufacturing,mining tonmile/MW
6,gas,front_need,aluminum,204.0000,"(39.915376, 115.46497)",1494.313641
7,hydro,front_need,aluminum,1585.2096,"(38.395595, 140.401897)",6031.770512
8,coal,front_need,aluminum,419.0000,"(39.915376, 115.46497)",1925.938033
9,wind,front_need,aluminum,8026.8000,"(38.395595, 140.401897)",49700.645724
10,solar,front_need,aluminum,19000.0000,"(38.395595, 140.401897)",101905.295838
...,...,...,...,...,...,...
229,nuclear,front_need,titanium,0.0100,"(39.915376, 115.46497)",0.045378
233,solar,oper_need,vegetable_oil,6001.0000,"(38.395595, 140.401897)",63411.037057
238,wind,front_need,zinc,26.9824,"(39.915376, 115.46497)",94.283723
239,nuclear,front_need,zinc,2.0200,"(39.915376, 115.46497)",7.058420


# Importing plants data

In [5]:
plants = pd.read_csv(cwd+"//data//smart_plants_modified.csv")
plants

,Unnamed: 0,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
0,0.0,-1.0,Recommended Plant,hydro,68027.7,1.0,2020.0,12.0,2060.0,"['(39.814905,-122.332335)']",40.0
1,0.0,-1.0,Recommended Plant,hydro,1261.9,1.0,2021.0,12.0,2061.0,"['(39.814905,-122.332335)']",40.0
2,0.0,-1.0,Recommended Plant,gas,76.49945538335096,1.0,2050.0,12.0,2080.0,"['(39.719364,-76.161625)']",30.0
3,0.0,-1.0,Recommended Plant,gas,102.61736489317134,1.0,2050.0,12.0,2080.0,"['(29.64536,-95.45169)']",30.0
4,0.0,-1.0,Recommended Plant,gas,102.61736489317134,1.0,2050.0,12.0,2080.0,"['(26.451111,-98.1775)']",30.0
...,...,...,...,...,...,...,...,...,...,...,...
47089,NaN,NaN,SR_Paris_SRPAR,solar,6.8,10.0,2022.0,10.0,2047.0,"['(36.445894,-88.327688)']",25.0
47090,NaN,NaN,SR_Jackson_II_SRJA2,solar,1,8.0,2022.0,8.0,2047.0,"['(35.607757,-88.917046)']",25.0
47091,NaN,NaN,Lumina_II_Solar_Project_LUMII,solar,357.8,6.0,2023.0,6.0,2048.0,"['(32.883528,-100.746389)']",25.0
47092,NaN,NaN,Lumina_Solar_Project_LUMIN,solar,357.8,6.0,2023.0,6.0,2048.0,"['(32.901081,-100.982695)']",25.0


In [6]:
# Pre-processing plants
# Namely, dropping Unnamed: 12 and other irrelevant columns
plants = plants.drop(
        columns=[
            "Unnamed: 0", 
            "Plant Code", 
            "name", 
            "Operating Month", 
            "Planned Retirement Month",
            "plant_life"]
    )
plants["coordinate"] = plants["coordinate"].apply(lambda tup_s: tuple(float(x) for x in tup_s[3:-3].split(',')))
plants

,source,capacity,Operating Year,Planned Retirement Year,coordinate
0,hydro,68027.7,2020.0,2060.0,"(39.814905, -122.332335)"
1,hydro,1261.9,2021.0,2061.0,"(39.814905, -122.332335)"
2,gas,76.49945538335096,2050.0,2080.0,"(39.719364, -76.161625)"
3,gas,102.61736489317134,2050.0,2080.0,"(29.64536, -95.45169)"
4,gas,102.61736489317134,2050.0,2080.0,"(26.451111, -98.1775)"
...,...,...,...,...,...
47089,solar,6.8,2022.0,2047.0,"(36.445894, -88.327688)"
47090,solar,1,2022.0,2047.0,"(35.607757, -88.917046)"
47091,solar,357.8,2023.0,2048.0,"(32.883528, -100.746389)"
47092,solar,357.8,2023.0,2048.0,"(32.901081, -100.982695)"


In [7]:
# Declaring emission constant
emission_k = 161.8
merged = pd.merge(need_materials, plants, on=["source"])
merged

,source,stream,material,amount,manufacturing,mining tonmile/MW,capacity,Operating Year,Planned Retirement Year,coordinate
0,gas,front_need,aluminum,204.000,"(39.915376, 115.46497)",1494.313641,76.49945538335096,2050.0,2080.0,"(39.719364, -76.161625)"
1,gas,front_need,aluminum,204.000,"(39.915376, 115.46497)",1494.313641,102.61736489317134,2050.0,2080.0,"(29.64536, -95.45169)"
2,gas,front_need,aluminum,204.000,"(39.915376, 115.46497)",1494.313641,102.61736489317134,2050.0,2080.0,"(26.451111, -98.1775)"
3,gas,front_need,aluminum,204.000,"(39.915376, 115.46497)",1494.313641,102.61736489317134,2050.0,2080.0,"(43.094, -93.292222)"
4,gas,front_need,aluminum,204.000,"(39.915376, 115.46497)",1494.313641,102.61736489317134,2050.0,2080.0,"(40.0195, -105.202)"
...,...,...,...,...,...,...,...,...,...,...
556249,nuclear,oper_need,zirconium,804.168,"(38.395595, 140.401897)",3857.189424,77,2030.0,2090.0,"(43.519585, -112.046184)"
556250,nuclear,oper_need,zirconium,804.168,"(38.395595, 140.401897)",3857.189424,77,2030.0,2090.0,"(43.519585, -112.046184)"
556251,nuclear,oper_need,zirconium,804.168,"(38.395595, 140.401897)",3857.189424,77,2030.0,2090.0,"(43.519585, -112.046184)"
556252,nuclear,oper_need,zirconium,804.168,"(38.395595, 140.401897)",3857.189424,77,2030.0,2090.0,"(43.519585, -112.046184)"


# Merging plants with need_materials based on source

In [8]:
# Calculate the manufacturing transportation distance
merged["manufacturing tonmile/MW"] = (merged["amount"] / 1000) * merged.apply(lambda row: geodesic(row["coordinate"], row["manufacturing"]).miles, axis=1)

In [9]:
merged["total_emission"] = emission_k * float(merged["capacity"][0]) * (merged["mining tonmile/MW"] + merged["manufacturing tonmile/MW"])
merged = merged.drop(
        columns=[
            "manufacturing tonmile/MW", 
            "mining tonmile/MW", 
            "amount", 
            "coordinate", 
            "manufacturing", 
            "material"
        ],
        errors='ignore'
    )
merged

,source,stream,capacity,Operating Year,Planned Retirement Year,total_emission
0,gas,front_need,76.49945538335096,2050.0,2080.0,3.592279e+07
1,gas,front_need,102.61736489317134,2050.0,2080.0,3.680534e+07
2,gas,front_need,102.61736489317134,2050.0,2080.0,3.712938e+07
3,gas,front_need,102.61736489317134,2050.0,2080.0,3.476347e+07
4,gas,front_need,102.61736489317134,2050.0,2080.0,3.456475e+07
...,...,...,...,...,...,...
556249,nuclear,oper_need,77,2030.0,2090.0,9.955450e+07
556250,nuclear,oper_need,77,2030.0,2090.0,9.955450e+07
556251,nuclear,oper_need,77,2030.0,2090.0,9.955450e+07
556252,nuclear,oper_need,77,2030.0,2090.0,9.955450e+07


In [10]:
merged["Years"] = merged.apply(lambda r: [year for year in range(int(r['Operating Year']), int(r['Planned Retirement Year']) + 1)], axis=1)
merged = merged.drop(
    columns=[
        "Operating Year",
        "Planned Retirement Year"
    ],
    errors='ignore'
)
merged

,source,stream,capacity,total_emission,Years
0,gas,front_need,76.49945538335096,3.592279e+07,"[2050, 2051, 2052, 2053, 2054, 2055, 2056, 205..."
1,gas,front_need,102.61736489317134,3.680534e+07,"[2050, 2051, 2052, 2053, 2054, 2055, 2056, 205..."
2,gas,front_need,102.61736489317134,3.712938e+07,"[2050, 2051, 2052, 2053, 2054, 2055, 2056, 205..."
3,gas,front_need,102.61736489317134,3.476347e+07,"[2050, 2051, 2052, 2053, 2054, 2055, 2056, 205..."
4,gas,front_need,102.61736489317134,3.456475e+07,"[2050, 2051, 2052, 2053, 2054, 2055, 2056, 205..."
...,...,...,...,...,...
556249,nuclear,oper_need,77,9.955450e+07,"[2030, 2031, 2032, 2033, 2034, 2035, 2036, 203..."
556250,nuclear,oper_need,77,9.955450e+07,"[2030, 2031, 2032, 2033, 2034, 2035, 2036, 203..."
556251,nuclear,oper_need,77,9.955450e+07,"[2030, 2031, 2032, 2033, 2034, 2035, 2036, 203..."
556252,nuclear,oper_need,77,9.955450e+07,"[2030, 2031, 2032, 2033, 2034, 2035, 2036, 203..."


In [51]:
filtered = merged[merged['Years'].apply(lambda x: 2020 >= x[0] and x[len(x) - 1] <= 2050)]
filtered['Years'] = filtered['Years'].apply(lambda x: [yr for yr in x if yr >= 2020 and yr <= 2050])
filtered

/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_61461/4231370969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Years'] = filtered['Years'].apply(lambda x: [yr for yr in x if yr >= 2020 and yr <= 2050])


,source,stream,capacity,total_emission,Years
3427,gas,front_need,170.1,3.699207e+07,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
3428,gas,front_need,170.1,3.699207e+07,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
3429,gas,front_need,195.2,3.699207e+07,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
3430,gas,front_need,170.1,3.699207e+07,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
3431,gas,front_need,170.1,3.699207e+07,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
...,...,...,...,...,...
556241,nuclear,oper_need,"1,354.30",1.129398e+08,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
556242,nuclear,oper_need,"1,035.90",1.136991e+08,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
556245,nuclear,oper_need,801,1.070714e+08,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
556246,nuclear,oper_need,902.5,1.105728e+08,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."


In [62]:
years_of_interest = [(x, y, z) for x in range(2020, 2051) for y in ["front_need", "oper_need"] for z in ["coal", "gas", "nuclear", "solar", "wind", "hydro"]] 
years_of_interest_dict = {}
for item in years_of_interest:
    years_of_interest_dict[item] = 0.0

x = pd.DataFrame({"Emissions" : years_of_interest_dict})
x

Emissions
2020 front_need coal           0.0
                gas            0.0
                hydro          0.0
                nuclear        0.0
                solar          0.0
...                            ...
2050 oper_need  gas            0.0
                hydro          0.0
                nuclear        0.0
                solar          0.0
                wind           0.0

[372 rows x 1 columns]

In [74]:
x_lst = [(stream, source, emi, yrs) for stream, source, emi, yrs in zip(filtered['stream'], filtered['source'], filtered["total_emission"], filtered['Years'])]

In [77]:
for item in x_lst:
    stream = item[0]
    source = item[1]
    emi = item[2]
    yrs = item[3]
    for yr in yrs:
        x["Emissions"][yr][stream][source] = emi

In [78]:
x

Emissions
2020 front_need coal     6.935824e+09
                gas      4.534636e+09
                hydro    9.045126e+09
                nuclear  2.609406e+05
                solar    5.400435e+07
...                               ...
2050 oper_need  gas      2.300964e+10
                hydro    0.000000e+00
                nuclear  1.104550e+08
                solar    0.000000e+00
                wind     0.000000e+00

[372 rows x 1 columns]

In [ ]:
x.to_csv("emissions_smart.csv")

In [ ]:
for item in lst:
    stream = item[0]
    source = item[1]
    cap = item[2]
    emi = item[3]
    yrs = item[4]
    for yr in yrs:
        if yr > 2050:
            break
        if yr < 2020:
            continue

In [11]:
lst = [(stream, source, cap, emi, yrs) for stream, source, cap, emi, yrs in zip(merged['stream'], merged['source'], merged['capacity'], merged["total_emission"], merged['Years'])]

In [15]:
def multi_func(item):
    stream = item[0]
    source = item[1]
    cap = item[2]
    emi = item[3]
    yrs = item[4]
    for yr in yrs:
        if yr > 2050 or yr < 2020: 
            continue
        emissions["Emissions"][yr][stream][source] += emi

with mp.Pool(10) as pool:
    processed = pool.map(multi_func, lst)

In [16]:
emissions[:30]

Emissions
2020 front_need coal           0.0
                gas            0.0
                hydro          0.0
                nuclear        0.0
                solar          0.0
                wind           0.0
     oper_need  coal           0.0
                gas            0.0
                hydro          0.0
                nuclear        0.0
                solar          0.0
                wind           0.0
2021 front_need coal           0.0
                gas            0.0
                hydro          0.0
                nuclear        0.0
                solar          0.0
                wind           0.0
     oper_need  coal           0.0
                gas            0.0
                hydro          0.0
                nuclear        0.0
                solar          0.0
                wind           0.0
2022 front_need coal           0.0
                gas            0.0
                hydro          0.0
                nuclear        0.0
                solar          0.0
                wind           0.0

# Plan
1. Populate NaN fields in materials table -- ✅ 
2. Figure out what materials are required for each source, filtered by oper_need and front_need only. -- ✅ 
3. Figure out for each plant, total distance (including mining -> manufacturing -> plant), total kg of materials necessary to get all the materials per source
4. total kg (in tonns now) * Total distance (in miles) = you get total ton-mile per plant
5. total CO2 emission = 161.8 * ton-mile
6. For each year from 2020 to 2050, it requires one time delivery of front_need and remaining years oper_need
7. We get [Year | Source | Total Emission | Capacity for that Year of all stations based on source]

In [29]:
emissions.to_csv("emissions_smart.csv")